In [1]:
# loading packages
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_score, train_test_split 

In [2]:
# loading data
df = pd.read_csv("heart.csv", sep=',')

In [ ]:
# checking for null
df.isnull().sum(axis = 0)

In [3]:
# split
X = df.drop("target",axis=1)
y = df["target"].values


In [4]:
# create dummies
#TODO: Try this way
X = pd.get_dummies(X, columns = ['cp','thal','slope'])

In [ ]:
X.head(10)

In [5]:
#Scaling for SVM
#TODO:
#check if all columns should be scalled
 
from sklearn import preprocessing
standard_scaler = preprocessing.StandardScaler()
X_standard = standard_scaler.fit_transform(X)

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
svm = SVC()
scores = cross_val_score(svm, X_standard, y, cv=5)

In [ ]:
 print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [7]:
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials

In [13]:
def objectiveSVM(params):
    del params['type']
    clf = SVC(**params)
    accurancy = cross_val_score(clf, X_standard, y, cv = 5).mean()
    return{'loss': - accurancy, 'status':STATUS_OK}


In [9]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'naive_bayes',
    },
    {
        'type': 'svm',
        'C': hp.lognormal('C', 0, 1.0),
        'kernel': hp.choice('kernel', ['linear', 'rbf'])
    },
])

In [14]:
trails = Trials()
algo=tpe.suggest
best_result = fmin(
    fn=objectiveSVM, 
    space=search_space,
    algo=algo,
    max_evals=16,
    trials=trails)

100%|██████████| 16/16 [00:00<00:00, 25.98trial/s, best loss: -0.8282513661202187]


In [15]:
print(best_result)

{'classifier_type': 0}
